# From Neurons to Networks: Exploring Deep Learning Algorithms

Roderick Perez, Ph.D.
OMV

# What are PINNs?

Physics-Informed Neural Networks (PINNs) are a special type of neural network that leverages prior knowledge of physics (typically in the form of differential equations) to guide the learning process. This helps the neural network learn more effectively when data is scarce or noisy by regularizing the network to follow known physical laws.

## Why Use PINNs?

* **Regularization through physics**: PINNs reduce overfitting in situations where data is limited or noisy by embedding physical constraints (through differential equations) into the network's loss function.
* **Real-world application**: This is particularly useful for modeling physical systems where collecting data is hard but the governing equations are known.

Example: Cooling of a Coffee Cup
Let’s consider a simple real-world example of a cooling coffee cup. This system follows Newton’s Law of Cooling:

In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd

# Neural Network Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        self.r = nn.Parameter(torch.tensor([0.005]))  # Cooling rate parameter
    
    def forward(self, t):
        x = torch.relu(self.fc1(t))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Physics loss: PINN loss based on Newton's Law of Cooling
def physics_loss(model, Tenv=25.0, ts=torch.linspace(0, 1000, steps=1000).view(-1,1)):
    ts.requires_grad_(True)
    temps = model(ts)
    
    # Compute gradient (derivative of temperature with respect to time)
    dT_dt = autograd.grad(temps, ts, grad_outputs=torch.ones_like(temps), create_graph=True)[0]
    
    # Compute the physics-based loss (ODE)
    ode = dT_dt + model.r * (temps - Tenv)
    
    # Return mean squared error of the ODE
    return torch.mean(ode**2)

# Data Loss (MSE)
def data_loss(predicted, actual):
    return torch.mean((predicted - actual)**2)

# Training the PINN
def train_pinn(model, data, epochs=1000):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        model.train()

        # Data Loss
        pred = model(data["time"])
        loss_data = data_loss(pred, data["temp"])

        # Physics Loss
        loss_phys = physics_loss(model)

        # Total Loss
        loss = loss_data + loss_phys
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item()}")
